In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
from torch.utils.data import Dataset
import json
import torch


device= "cuda" if torch.cuda.is_available() else "cpu"

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):  # the path is the finetuning dataset path
        self.data= json.load(open(path, "r"))
        self.X= []

        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx]= "<startofstring>"+i+"<bot>: "+self.X[idx+1]+"<endofstring>"
            except:
                break

        self.X= self.X[:3000]
        print(self.X[0])

        self.X_encoded= tokenizer(self.X, max_length=50, truncation=True, padding= "max_length", return_tensors="pt")
        self.input_ids= self.X_encoded['input_ids']
        self.attention_mask= self.X_encoded['attention_mask']



    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [5]:
model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model= model.to(device)


tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<startoftoken>",
                              "eos_token": "<endofstring>"})

tokenizer.add_tokens({"<bot>: "})
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50261. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(50261, 768)

In [6]:
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):
    epochs= 10

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X= X.to(device)
            a= a.to(device)

            optim.zero_grad()
            loss= model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        infer("hello there?")

In [24]:
def infer(inp):
    inp= "<startoftoken> "+inp+"<bot>: "
    inp= tokenizer(inp, return_tensors="pt")
    X= inp["input_ids"].to(device)
    a= inp["attention_mask"].to(device)
    output= model.generate(X, attention_mask=a)
    output= tokenizer.decode(output[0])
    return output

In [8]:
from torch.optim import Adam

chatData= ChatData("chat_data.json", tokenizer)
chatData= DataLoader(chatData, batch_size=64)
model.train()
optim= Adam(model.parameters(), lr=1e-3)

<startofstring>I love iphone! i just bought new iphone!<bot>: Thats good for you, i'm not very into new tech<endofstring>


In [9]:
train(chatData, model, optim)

100%|██████████| 10/10 [04:41<00:00, 28.19s/it]


In [26]:
print(infer("how are ou doing?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startoftoken> how are ou doing?<bot>:  I am doing great, how are you?


In [27]:
while True:
  inp= input()
  print(infer(inp))

wass up


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<startoftoken> wass up<bot>:  I am a student <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
which department of student?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startoftoken> which department of student?<bot>:  I am a student of university <endofstring> <pad> <pad> <pad>
which university?


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startoftoken> what expericence?<bot>:  I am a graduate student. <endofstring> <pad> <pad> <pad>
from which college?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startoftoken> from which college?<bot>:  I am a student of a college<bot>: I


KeyboardInterrupt: ignored